Take a look at the [repository](https://github.com/mit-han-lab/gan-compression) for more information

# Install

In [0]:
!git clone https://github.com/mit-han-lab/gan-compression.git

In [0]:
import os
os.chdir('gan-compression')
print(os.getcwd())

In [0]:
!pip install -r requirements.txt
!pip install --upgrade git+https://github.com/mit-han-lab/torchprofile.git

In [0]:
import pickle
import time
import tqdm

import numpy as np
import torch

# Datasets
Download the dataset edges2shoes for testing.

In [0]:
!bash datasets/download_pix2pix_dataset.sh edges2shoes-v

# Pretrained Models
Download the original model and our compressed of edges2shoes dataset.

In [0]:
!python scripts/download_model.py --model pix2pix --task edges2shoes-r --stage full
!python scripts/download_model.py --model pix2pix --task edges2shoes-r --stage compressed
print('Download the pretrained models successfully!!!')

# Test

Get the options for the test.

In [0]:
!python get_test_opt.py --dataroot database/edges2shoes-v \
--results_dir results-pretrained/pix2pix/edges2shoes-r/full \
--ngf 64 --netG resnet_9blocks \
--restore_G_path pretrained/pix2pix/edges2shoes-r/full/latest_net_G.pth \
--real_stat_path real_stat/edges2shoes-r_B.npz \
--need_profile --num_test 0

!python get_test_opt.py --dataroot database/edges2shoes-v \
--results_dir results-pretrained/pix2pix/edges2shoes-r/compressed \
--restore_G_path pretrained/pix2pix/edges2shoes-r/compressed/latest_net_G.pth \
--config_str 32_32_48_32_48_48_16_16 \
--real_stat_path real_stat/edges2shoes-r_B.npz \
--need_profile --num_test 0

Get the dataloader for evaluation.

In [0]:
from data import create_dataloader
with open('opts/opt_full.pkl', 'rb') as f:
    opt = pickle.load(f)
dataloader = create_dataloader(opt)

Get the original full model.

In [0]:
from models import create_model

with open('opts/opt_full.pkl', 'rb') as f:
    opt = pickle.load(f)
model_full = create_model(opt, verbose=False)
model_full.setup(opt, verbose=False)

Get our compressed model.

In [0]:
with open('opts/opt_compressed.pkl', 'rb') as f:
    opt = pickle.load(f)
model_ours = create_model(opt, verbose=False)
model_ours.setup(opt, verbose=False)

Get visual results of the models.

In [0]:
from IPython.display import display
from PIL import Image
import torchvision.transforms as transforms

from utils.util import save_image, tensor2im

transform_list = [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
transform = transforms.Compose(transform_list)
print('             Input                        Original                      Compressed                Ground-truth')
A_dir = 'imgs/edges2shoes/A'
B_dir = 'imgs/edges2shoes/B'
files = os.listdir(A_dir)
for file in files:
    if not file.endswith('.png'):
        continue
    base = file.split('.')[0]
    A_path = os.path.join(A_dir, file)
    B_path = os.path.join(B_dir, file)
    A_img = Image.open(A_path).convert('RGB')
    input = transform(A_img).to('cuda:0')
    input = input.reshape([1, 3, 256, 256])
    output_full = model_full.netG(input).cpu()
    output_ours = model_ours.netG(input).cpu()
    B_full = tensor2im(output_full)
    B_ours = tensor2im(output_ours)
    save_image(B_full, 'output/full/%s.png' % base, create_dir=True)
    save_image(B_ours, 'output/ours/%s.png' % base, create_dir=True)
    p1 = Image.open(A_path)
    p2 = Image.open('output/full/%s.png' % base)
    p3 = Image.open('output/ours/%s.png' % base)
    p4 = Image.open(os.path.join(B_path))
    all = Image.new('RGB', (256*4, 256))
    all.paste(p1, (0, 0))
    all.paste(p2, (256, 0))
    all.paste(p3, (256*2, 0))
    all.paste(p4, (256*3, 0))
    display(all)

Profile and evaluate the two models.

In [0]:
def evaluate(model):
    fakes = []
    for i, data in enumerate(tqdm.tqdm(dataloader)):
        model.set_input(data)
        if i == 0:
            macs, params = model.profile(verbose=False)
        model.test()
        visuals = model.get_current_visuals()
        generated = visuals['fake_B'].cpu()
        fakes.append(generated)
    # Test the latency of the model
    for i in range(100):
        model.test()
        torch.cuda.synchronize()
    start_time = time.time()
    for i in range(100):
        model.test()
        torch.cuda.synchronize()
    cost = time.time()-start_time
    latency = cost/100
    return macs, params, latency, fakes

macs_full, params_full, latency_full, fakes_full = evaluate(model_full)
macs_ours, params_ours, latency_ours, fakes_ours = evaluate(model_ours)
print('\n\n')
print('Full: %.3fG MACs\t%.3fM Params\t%.5fs Latency' % (macs_full/1e9, params_full/1e6, latency_full))
print('Ours: %.3fG MACs\t%.3fM Params\t%.5fs Latency' % (macs_ours/1e9, params_ours/1e6, latency_ours))

Download the statistical information of the ground-truth images for FID computation.

In [0]:
!bash ./datasets/download_real_stat.sh edges2shoes-r B
npz = np.load('real_stat/edges2shoes-r_B.npz')
print('Loaded the statistical information of the ground-truth images.')

Define the Inception Model for FID computation. (**This may take about 3mins.**)


In [0]:
from metric.inception import InceptionV3
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
inception_model = InceptionV3([block_idx])
inception_model.to('cuda:0')
inception_model.eval()
print('Successfully built an InceptionV3 model.')

Computing the FID of the two models.

In [0]:
from metric import get_fid
fid_full = get_fid(fakes_full, inception_model, npz, 'cuda:0', 16)
fid_ours = get_fid(fakes_ours, inception_model, npz, 'cuda:0', 16)
print('\n\n')
print('Full FID: %.3f' % fid_full)
print('Ours FID: %.3f' % fid_ours)